In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '2'

In [19]:
from transformers import AutoFeatureExtractor, AutoModel, AutoTokenizer
import librosa
import torch
from glob import glob
from jiwer import cer
from tqdm import tqdm
import json
import os

torch.set_grad_enabled(False)

In [3]:
from datasets import load_dataset

ds = load_dataset("malaysia-ai/common_voice_17_0")

In [4]:
df_test = ds['test'].to_pandas()

In [5]:
df_test.shape

(533642, 14)

In [6]:
model_id = "mesolitica/whisper-conv-VQ-32k-large-v3-turbo"
feature_extractor = AutoFeatureExtractor.from_pretrained(model_id)
model = AutoModel.from_pretrained(model_id, trust_remote_code = True, torch_dtype = 'auto').cuda()
tokenizer = AutoTokenizer.from_pretrained(model_id)

preprocessor_config.json:   0%|          | 0.00/357 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

whispervq_conv.py: 0.00B [00:00, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/mesolitica/whisper-conv-VQ-32k-large-v3-turbo:
- whispervq_conv.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.


model.safetensors:   0%|          | 0.00/3.40G [00:00<?, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

normalizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

special_tokens_map.json: 0.00B [00:00, ?B/s]

In [7]:
!mkdir evaluate-whisper-conv-VQ-32k-large-v3-turbo

In [16]:
from tqdm import tqdm
import json
import os

template = '<|startoftranscript|><|{locale}|><|transcribe|><|notimestamps|>'

for i in tqdm(reversed(range((len(df_test) // 2) * 1, (len(df_test) // 2) * 2, 1))):
    
    filename = f'evaluate-whisper-conv-VQ-32k-large-v3-turbo/{i}.json'
    try:
        if os.path.exists(filename):
            with open(filename) as fopen:
                json.load(fopen)
                continue
    except:
        pass
    
    t = df_test['sentence'].iloc[i]

    if not isinstance(t, str):
        continue

    if len(t) < 5:
        continue

    l = df_test['locale'].iloc[i]

    if not isinstance(l, str):
        continue

    if len(l) < 2:
        continue
            
    y, sr = librosa.load(df_test.iloc[i]['audio_filename'], sr = feature_extractor.sampling_rate)
    input_ids = tokenizer(template.replace('{locale}', l), 
        add_special_tokens = False, return_tensors = 'pt')['input_ids']
    features = feature_extractor(
        [y], 
        return_tensors = 'pt', 
        return_attention_mask = True,
        sampling_rate = 16000,
    )
    features['input_features'] = features['input_features'].cuda()
    features['attention_mask'] = features['attention_mask'].cuda()
    features['decoder_input_ids'] = input_ids.cuda()
    generate_kwargs = dict(
        **features,
        max_new_tokens=256,
        temperature=0.1,
        do_sample=True
    )
    generation_output = model.generate(**generate_kwargs)
    decoded = tokenizer.decode(generation_output[0])
    with open(filename, 'w') as fopen:
        json.dump({'predict': decoded, 'actual': t}, fopen)

266821it [1:10:07, 63.41it/s]    


In [20]:
from collections import defaultdict
import string

def clean(s):
    s = ''.join([c for c in s.lower() if c not in string.punctuation])
    return s.strip()

languages = defaultdict(list)

files = glob('evaluate-whisper-conv-VQ-32k-large-v3-turbo/*')
for f in tqdm(files):
    with open(f) as fopen:
        d = json.load(fopen)
    l = d['predict'].split('<|')[2].split('|>')[0]
    predict = d['predict'].split('<|notimestamps|>')[1].replace('<|endoftext|>', '')
    actual = d['actual']
    predict = clean(predict)
    actual = clean(actual)
    score = cer(actual, predict)
    languages[l].append(score)

len(files)

100%|██████████████████████████████████████████████████████████████████████████████| 532364/532364 [00:31<00:00, 16755.00it/s]


532364

In [21]:
import numpy as np

means = []
for k, v in languages.items():
    mean = np.mean(v)
    if mean >= 1.0:
        mean = 1.0
    print(f'lang: {k}, samples: {len(v)}, CER: {mean}')
    means.append(mean)
    
print('\naverage CER:', np.mean(means))

lang: gl, samples: 9949, CER: 0.179959138002185
lang: en, samples: 16379, CER: 0.18379318688291674
lang: ar, samples: 10458, CER: 0.442098309275015
lang: kab, samples: 14972, CER: 0.5076671557224989
lang: ml, samples: 703, CER: 0.7163533887103217
lang: kk, samples: 514, CER: 0.4628507980881995
lang: ltg, samples: 2904, CER: 0.4205037910930837
lang: fr, samples: 16145, CER: 0.1654582834310926
lang: de, samples: 16170, CER: 0.1724037776794763
lang: fi, samples: 1554, CER: 0.39460069431609057
lang: pt, samples: 9432, CER: 0.17848626855907665
lang: ia, samples: 1816, CER: 0.15100353821127493
lang: eu, samples: 13621, CER: 0.296558713578422
lang: ro, samples: 3896, CER: 0.23927959309114133
lang: sw, samples: 12086, CER: 0.37654191236655504
lang: sv-SE, samples: 5247, CER: 0.30514152709793296
lang: ta, samples: 8263, CER: 0.48104527337629344
lang: et, samples: 2653, CER: 0.5043074385490827
lang: lg, samples: 11902, CER: 0.40869471032709415
lang: it, samples: 15154, CER: 0.1520149858391226
la

In [22]:
len(means)

115